In [ ]:
#!pip install polars

In [ ]:
import pandas as pd
import numpy as np
import polars as pl
import ast

In [ ]:
### Read csv file
df = pd.read_csv('./csv/nrw_hai_extracted.csv')

In [ ]:
df.nunique()

In [ ]:
len(df)

In [ ]:
#x_df=df.groupby(['AGG_DAY_PERIOD', 'BOUNDS'], as_index=False).mean()
del x_df

In [ ]:
# Group by AGG_DAY_PERIOD and BOUNDS and compute the mean ACTIVITY_INDEX_TOTAL
grouped_df = df.groupby(['AGG_DAY_PERIOD', 'BOUNDS'], as_index=False).agg({'ACTIVITY_INDEX_TOTAL': 'mean'})#.rename(columns={'ACTIVITY_INDEX_TOTAL': 'MEAN_ACTIVITY_INDEX_TOTAL'})

In [ ]:
grouped_df.head(100000000000)[50000000:50000009]

In [ ]:
grouped_df.nunique()

In [ ]:
len(grouped_df['ACTIVITY_INDEX_TOTAL'])
#grouped_df.to_csv('./csv/grouped_csv.csv')

In [ ]:
df=pl.read_csv('./csv/grouped_csv.csv')

In [ ]:
df=pl.DataFrame(df)
df.head()

In [ ]:
# Function to calculate the centroid of the bounds
def calculate_centroid(bounds):
    lower_left_lon, lower_left_lat, upper_right_lon, upper_right_lat = bounds
    centroid_lon = (lower_left_lon + upper_right_lon) / 2
    centroid_lat = (lower_left_lat + upper_right_lat) / 2
    return centroid_lat, centroid_lon

In [ ]:
# Parse the BOUNDS field and calculate the centroid for each bound
centroids = df['BOUNDS'].map_elements(lambda x: calculate_centroid(ast.literal_eval(x)), return_dtype=pl.List(pl.Float64)).alias("CENTROID")

In [ ]:
#df.write_csv('./csv/grouped_csv_centroid')

In [ ]:
# Convert the centroids into separate columns
centroid_lat = [c[0] for c in centroids]
centroid_lon = [c[1] for c in centroids]

In [ ]:
# Add the centroid columns to the DataFrame
df = df.with_columns([
    pl.Series(name="CENTROID_LAT", values=centroid_lat),
    pl.Series(name="CENTROID_LON", values=centroid_lon)
])

In [ ]:
# Save the resulting DataFrame to a new CSV file
df.write_csv('centroid_bounds.csv')

print("The CSV file with centroids has been created successfully.")

In [ ]:
# Parse the BOUNDS field and calculate the centroid for each bound
grouped_df['BOUNDS'] = grouped_df['BOUNDS'].apply(eval)  # Convert the string representation of the list to an actual list

In [ ]:
grouped_df[['CENTROID_LAT', 'CENTROID_LON']] = grouped_df['BOUNDS'].apply(lambda b: pd.Series(calculate_centroid(b)))

### Try on chunks to resolve memory error

In [1]:
import pandas as pd
import ast

In [2]:
# Load the CSV file in chunks to handle large data efficiently
chunksize = 2500000
chunks = pd.read_csv('./csv/nrw_hai_extracted.csv', chunksize=chunksize)

# Initialize an empty DataFrame to hold the processed data
result_df = pd.DataFrame()

In [3]:
# Function to calculate the centroid of the bounds
def calculate_centroid(bounds):
    lower_left_lon, lower_left_lat, upper_right_lon, upper_right_lat = bounds
    centroid_lon = (lower_left_lon + upper_right_lon) / 2
    centroid_lat = (lower_left_lat + upper_right_lat) / 2
    return centroid_lat, centroid_lon

In [5]:
# Process each chunk
for chunk in chunks:
    # Extract the required fields
    extracted_chunk = chunk[['AGG_DAY_PERIOD', 'BOUNDS', 'ACTIVITY_INDEX_TOTAL']].copy()
    
    # Parse the BOUNDS field and calculate the centroid for each bound
    extracted_chunk['BOUNDS'] = extracted_chunk['BOUNDS'].apply(lambda x: tuple(ast.literal_eval(x)))  # Safely parse the list from string and convert to tuple
    extracted_chunk[['CENTROID_LAT', 'CENTROID_LON']] = extracted_chunk['BOUNDS'].apply(lambda b: pd.Series(calculate_centroid(b)))
    
    # Group by AGG_DAY_PERIOD and BOUNDS and compute the mean ACTIVITY_INDEX_TOTAL
    grouped_chunk = extracted_chunk.groupby(['AGG_DAY_PERIOD', 'BOUNDS', 'CENTROID_LAT', 'CENTROID_LON'], as_index=False).agg({
        'ACTIVITY_INDEX_TOTAL': 'mean'
    }).rename(columns={'ACTIVITY_INDEX_TOTAL': 'MEAN_ACTIVITY_INDEX_TOTAL'})
    
    # Append the processed chunk to the result DataFrame
    result_df = pd.concat([result_df, grouped_chunk], ignore_index=True)

In [6]:
# Save the resulting DataFrame to a new CSV file
result_df.to_csv('mean_activity_index_per_day_bounds.csv', index=False)

print("The CSV file has been created successfully.")

The CSV file has been created successfully.


In [ ]:
result_df.nunique()